# Capter10 隐马尔可夫模型
## HMM模型
### 另外参考[umdhmm](http://www.kanungo.com/software/software.html#umdhmm)，[52nlp](https://www.52nlp.cn/category/hidden-markov-model)

In [1]:
import numpy as np

class HMM():
    def forward(self, PI, A, B, O):
        mid_prob = []
        N = A.shape[0]
        T = len(O)
        for t in range(T):
            if t == 0:
                mid_prob.append(PI * B[:, O[t]])
            else:
                mid_prob.append([])
                for i in range(N):
                    mid_prob[t].append(np.inner(mid_prob[t - 1], A[:, i])*B[i][O[t]])
        return np.sum(mid_prob[T - 1])

    def viterbi(self, PI, A, B, O):
        mid_prob = []
        stat_info = []
        N = A.shape[0]
        M = B.shape[1]
        T = len(O)
        for t in range (T):
            stat_info.append([])
            if t == 0:
                mid_prob.append(PI * B[:, O[0]])
            else:
                mid_prob.append([])
                for i in range(N):
                    s_p = mid_prob[t - 1] * A[:, i]
                    stat_info[t].append({'val':np.max(s_p), 'idx':np.argmax(s_p)})
                    mid_prob[t].append(stat_info[t][i]['val'] * B[i][O[t]])
        path = []
        path.append(np.argmax(mid_prob[T - 1]))
        for t in range(T - 2, -1, -1):
            path.append(stat_info[t+1][path[T-2-t]]['idx'])
        path.reverse()
        return path

    def forward_scale(self, PI, A, B, O):
        mid_prob = []
        scale = []
        N = A.shape[0]
        T = len(O)
        for t in range(T):
            if t == 0:
                mid_prob.append(PI * B[:, O[t]])
            else:
                mid_prob.append([])
                for i in range(N):
                    mid_prob[t].append(np.inner(mid_prob[t - 1], A[:, i])*B[i][O[t]])
            scale.append(np.sum(mid_prob[t]))
            mid_prob[t] = mid_prob[t] / scale[t]
        return (np.array(mid_prob), np.array(scale), np.sum(np.log(scale)))

    def backward_scale(self, PI, A, B, O, scale):
        mid_prob = []
        N = A.shape[0]
        T = len(O) - 1
        mid_prob.append(np.ones(N) * (1.0/scale[T]))
        for t in range (T, 0, -1):
            mid_prob.append([])
            for j in range(N):
                prob = np.sum(A[j, :] * B[:, O[t]] * mid_prob[T - t])
                mid_prob[T-t+1].append(prob / scale[t - 1])
        mid_prob.reverse()
        return np.array(mid_prob)

    def cal_gamma(self, O, alpha, beta):
        T = len(O)
        gamma = []
        for t in range (T):
            gamma.append(alpha[t] * beta[t])
            g_sum = np.sum(gamma[t])
            gamma[t] = gamma[t] / g_sum
        return np.array(gamma)

    def cal_xi(self, A, B, O, alpha, beta):
        xi = []
        T = len(O)
        for t in range(T - 1):
            res = []
            for i in range(A.shape[0]):
                res.append(alpha[t][i] * beta[t+1] * A[i, :] * B[:, O[t+1]])
            res_sum = np.sum(res)
            xi.append(res/res_sum)
        return np.array(xi)

    def BaumWelch(self, O, N, M, eps = 0.001):
        T = len(O)
        PI = np.random.rand(N)
        val_sum = np.sum(PI)
        PI = PI/val_sum
        A = np.random.rand(N, N)
        val_sum = np.sum(A, axis=1)
        for i in range(N):
            A[i] = A[i, :] / val_sum[i]
        B = np.random.rand(N, M)
        val_sum = np.sum(B, axis=1)
        for i in range(M):
            B[i] = B[i, :] / val_sum[i]

        old_prob = -1
        iter_num = 0
        while True:
            alpha, scale, prob = self.forward_scale(PI, A, B, O)
            if old_prob == -1:
                old_prob = prob
            else:
                if abs(prob - old_prob) < eps:
                    break
            beta = self.backward_scale(PI, A, B, O, scale)
            gamma =self.cal_gamma(O, alpha, beta)
            xi = self.cal_xi(A, B, O, alpha, beta)
            PI = 0.001 + 0.999 * gamma[0, :]
            A_sum = np.sum(gamma[:T-1, :], axis = 0)
            A_val = np.sum(xi[:T-1, :], axis = 0)
            B_sum = np.sum(gamma, axis = 0)
            for i in range(N):
                A[i] = 0.001 + 0.999 * A_val[i] / A_sum[i]
                for k in range(M):
                    B_val = 0.0
                    for t in range(T):
                        if O[t] == k:
                            B_val = B_val + gamma[t][i]
                    B[i][k] = 0.001 + 0.999 * B_val / B_sum[i]
            print("loop: %d, prob: %lf, prob_diff: %lf" % (iter_num, prob, abs(prob - old_prob)))
            iter_num = iter_num + 1
            old_prob = prob
        return (PI, A, B)

## 算法测试
### 简单测试用例

In [2]:
# 问题1:概率计算（使用第一版书本章例子）
A = np.array([[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]])
B = np.array([[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]])
PI = [0.2, 0.4, 0.4]
O = [0, 1, 0]
model = HMM()
print('概率：%f\n' % (model.forward(PI, A, B, O)))

# 问题2: 无监督学习
O = [0,0,0,0,1,0,1,1,1,1]
print('学习：')
print('原始参数概率：%f' % (model.forward(PI, A, B, O)))
PI1, A1, B1 = model.BaumWelch(O, 3, 2)
print("学习后参数：PI={}\n, A={}\n, B={}".format(PI1, A1, B1))
print('学习参数概率：%f\n' % (model.forward(PI1, A1, B1, O)))

# 问题3: 预测 （使用第一版书本章例子）
O = [0, 1, 0]
print('预测: ', [x+1 for x in model.viterbi(PI, A, B, O)])

概率：0.130218

学习：
原始参数概率：0.001025
loop: 0, prob: -7.511149, prob_diff: 0.000000
loop: 1, prob: -6.933003, prob_diff: 0.578146
loop: 2, prob: -6.862454, prob_diff: 0.070549
loop: 3, prob: -6.814729, prob_diff: 0.047725
loop: 4, prob: -6.766552, prob_diff: 0.048177
loop: 5, prob: -6.705045, prob_diff: 0.061507
loop: 6, prob: -6.617510, prob_diff: 0.087535
loop: 7, prob: -6.481943, prob_diff: 0.135567
loop: 8, prob: -6.255157, prob_diff: 0.226786
loop: 9, prob: -5.882904, prob_diff: 0.372253
loop: 10, prob: -5.398239, prob_diff: 0.484665
loop: 11, prob: -4.984953, prob_diff: 0.413286
loop: 12, prob: -4.731295, prob_diff: 0.253658
loop: 13, prob: -4.564830, prob_diff: 0.166465
loop: 14, prob: -4.429306, prob_diff: 0.135523
loop: 15, prob: -4.314001, prob_diff: 0.115306
loop: 16, prob: -4.223092, prob_diff: 0.090909
loop: 17, prob: -4.159117, prob_diff: 0.063975
loop: 18, prob: -4.118940, prob_diff: 0.040176
loop: 19, prob: -4.096003, prob_diff: 0.022938
loop: 20, prob: -4.083772, prob_diff:

### HMM在分词上的应用示例

In [3]:
from collections import namedtuple

train_raw = open('data/pku_training.utf8').read()

#begin:B, middle:M, end:E, single:S
Hide = namedtuple('Hide', 'B, M, E, S')
hide_stat = Hide(0, 1, 2, 3)
words = set(train_raw)
words.remove('\n')
words.remove(' ')
words = list(words)
words_dict = {words[i] : i for i, w in enumerate(words)}
words_idx_dict = {v : k for k, v in words_dict.items()}

def get_hmm_params():
    train_lines = train_raw.split('\n')
    PI_num = []
    A_num = []
    B_num = []
    for i in range(len(hide_stat)):
        PI_num.append(0)
        a = [0] * len(hide_stat)
        A_num.append(a)
        b = [0] * len(words)
        B_num.append(b)

    for l in train_lines:
        flds = l.split(' ')
        first = True
        lst_stat = None
        for fld in flds:
            if len(fld) == 1:
                cur_stat = hide_stat.S
                if first:
                    PI_num[cur_stat] += 1
                else:
                    A_num[lst_stat][cur_stat] += 1
                B_num[cur_stat][words_dict[fld]] += 1
                lst_stat = cur_stat
            else:
                if first:
                    PI_num[hide_stat.B] += 1
                w_list = list(fld)
                for i, w in enumerate(w_list):
                    if i == 0:
                        cur_stat = hide_stat.B
                    elif i == len(w_list) - 1:
                        cur_stat = hide_stat.E
                    else:
                        cur_stat = hide_stat.M
                    B_num[cur_stat][words_dict[w]] += 1
                    if lst_stat != None:
                        A_num[lst_stat][cur_stat] += 1
                    lst_stat = cur_stat
            first = False

    PI = np.array(PI_num)
    PI = PI/np.sum(PI)
    A = np.array(A_num)
    A = A/np.sum(A, axis=0)
    B = np.array(B_num)
    B = B/(np.sum(B, axis = 1).reshape([4,1]))
    return (PI, A, B)

def print_output(O, hides):
    output = ''
    for i, s in enumerate(hides):
        w = words_idx_dict[O[i]]
        if s == hide_stat.B:
            output += ' '+w
        elif s == hide_stat.M:
            output += w
        elif s == hide_stat.E:
            output += w+' '
        else:
            output += ' ' + w + ' '
    print(output)

inputs = ['我们希望，新世纪成为各国人民共享和平的世纪。在20世纪里，世界饱受各种战争和冲突的苦难。',
    '北京新年音乐会展现经典魅力尉健行李岚清与数千首都观众一起欣赏。',
    '本届冰雪节第一次升格为国际级冰雪盛会，来自20多个国家的使节云集冰城；盛会还吸引了美国、日本、俄罗斯等国的冰雪爱好者。']
PI, A, B = get_hmm_params()
model = HMM()
for test_str in inputs:
    O = [words_dict[w] for w in test_str]
    hides = model.viterbi(PI, A, B, O)
    print_output(O, hides)

 我们  希望  ，  新  世纪  成为  各国  人民  共享  和平  的  世纪  。  在20世纪  里  ，  世界  饱受  各种  战争  和  冲突  的  苦难  。 
 北京  新年  音乐会  展现  经典  魅力  尉  健行  李  岚清  与  数千  首  都  观众  一起  欣赏  。 
 本届  冰雪节  第一  次  升格  为  国际  级  冰雪  盛会  ，  来  自20多  个  国家  的  使节  云集  冰城  ；  盛会  还  吸引  了  美国  、  日本  、  俄罗斯等国  的  冰雪  爱  好者  。 
